In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine

In [2]:
with open('yelp_academic_dataset_business.json', 'r', encoding='utf-8') as f:
        business_data = [json.loads(line) for line in f]

In [3]:
business_df = pd.DataFrame(business_data)

In [4]:
with open('yelp_academic_dataset_checkin.json', 'r', encoding='utf-8') as f:
        checkin_data = [json.loads(line) for line in f]
checkin_df = pd.DataFrame(checkin_data)

In [5]:
chunk_size = 10000

review_data = []
review_dfs = []
with open('yelp_academic_dataset_review.json', 'r', encoding='utf-8') as f:
    for line in f:
        review_data.append(json.loads(line))
        if len(review_data) >= chunk_size:
            review_df_chunk = pd.DataFrame(review_data)
            review_dfs.append(review_df_chunk) 
            review_data = []

if review_data:
    review_df_chunk = pd.DataFrame(review_data)
    review_dfs.append(review_df_chunk) 
    
review_df = pd.concat(review_dfs)


In [6]:
with open('yelp_academic_dataset_tip.json', 'r', encoding='utf-8') as f:
        tip_data = [json.loads(line) for line in f]
tip_df = pd.DataFrame(tip_data)

In [7]:
chunk_size = 10000  

user_dfs = []
with open('yelp_academic_dataset_user.json', 'r', encoding='utf-8') as f:
    chunk_data = []
    for line in f:
        chunk_data.append(json.loads(line))
        if len(chunk_data) >= chunk_size:
            user_df_chunk = pd.DataFrame(chunk_data)
            user_dfs.append(user_df_chunk) 
            chunk_data = []

    if chunk_data:
        user_df_chunk = pd.DataFrame(chunk_data)
        user_dfs.append(user_df_chunk)

user_df = pd.concat(user_dfs)


In [8]:
print(business_df.shape)
print(checkin_df.shape)
print(review_df.shape)
print(tip_df.shape)
print(user_df.shape)

(150346, 14)
(131930, 2)
(6990280, 9)
(908915, 5)
(1987897, 22)


In [9]:
business_df.drop(['attributes','hours'],axis=1,inplace= True )

In [10]:
engine = create_engine('sqlite:///yelp.db')

# Define a function to load DataFrame into SQLite in chunks
def load_dataframe_chunked(df, table_name, engine, chunk_size=1000):
    with engine.connect() as connection:
        # Loop through DataFrame in chunks
        for i in range(0, len(df), chunk_size):
            chunk = df.iloc[i:i+chunk_size]
            # Use the 'append' method to append data to the table
            chunk.to_sql(table_name, con=connection, if_exists='append', index=False)

# List of DataFrames and corresponding table names
dataframes = [business_df, checkin_df, review_df, tip_df, user_df]
table_names = ['business', 'checkin', 'review', 'tip', 'user']

# Loop through each DataFrame and load it into SQLite in chunks
for df, table_name in zip(dataframes, table_names):
    load_dataframe_chunked(df, table_name, engine)
